# Importer les bibliothèques nécessaires

In [175]:
import pandas as pd
import ast 
import numpy as np
import csv

# Importer les données de Doctolib

In [176]:
data = pd.read_csv('data/extracted_data_final_0_1000.csv')

In [177]:
# Fonction pour réorganiser les profils cards
def reorder_cards(profile_cards):
    try:
        cards_dict = ast.literal_eval(profile_cards)
        
        reordered_cards = {
            'card_0': np.nan,
            'card_1': np.nan,
            'card_2': np.nan,
            'card_3': np.nan,
            'card_4': np.nan,
            'card_5': np.nan
        }

        def split_content(value):
            value = value.replace('\\n', '\n').replace('\\', '\n')
            return value.split('\n')

        for key, value in cards_dict.items():
            #print(key,value,'s')
            if value.startswith("Carte et informations d'accès"):
                #print('1')
                reordered_cards['card_1'] = split_content(value)
            elif value.startswith("Présentation"):
                #print('2')
                reordered_cards['card_2'] = split_content(value)
            elif value.startswith("Horaires et coordonnées"):
                #print('3')
                reordered_cards['card_3'] = split_content(value)
            elif value.startswith("Tarifs et remboursement") or value.startswith("Lieu"):
                #print('i')
                reordered_cards['card_0'] = split_content(value)
            elif value.startswith("Tarifs"):
                #print('4')
                reordered_cards['card_4'] = split_content(value)
            elif value.startswith("Informations légales"):
                #print('5')
                reordered_cards['card_5'] = split_content(value)
            else :
                if isinstance(value,list):
                    #print('i1')
                    reordered_cards['card_0'] = split_content(value)
        
        return reordered_cards
    except:
        return {
            'card_0': np.nan,
            'card_1': np.nan,
            'card_2': np.nan,
            'card_3': np.nan,
            'card_4': np.nan,
            'card_5': np.nan
        }

reordered_cards = data['Profile Cards'].apply(reorder_cards)

reordered_cards_df = pd.json_normalize(reordered_cards)

data_combined = pd.concat([data, reordered_cards_df], axis=1)

data_combined.drop(columns = 'Profile Cards', inplace=True)
print(data_combined[['Doctor Name', 'card_0', 'card_1', 'card_2', 'card_3', 'card_4', 'card_5']].tail(9))


                   Doctor Name  \
991        Dr Mathieu SABATIER   
992           Dr Eric LE FLOCH   
993    Agnès Montagnon -Thomas   
994     Dr Stella Danan-Hadida   
995  Dr Philippe Lazare SELLAM   
996          Dr Michèle Maizil   
997           Dr Thierry MEYER   
998          Dr Marc Stromboni   
999               Fabien GUIDE   

                                                card_0  \
991                                                NaN   
992  [Lieu 1, Lieu 2, Lieu 3, Radio Faidherbe, Radi...   
993  [Tarifs et remboursement, , , Tiers payant : S...   
994  [Tarifs et remboursement, , , , Conventionné s...   
995  [Lieu 1, Lieu 2, Cabinet du Docteur Philippe L...   
996                                                NaN   
997  [Tarifs et remboursement, , , , Conventionné, ...   
998  [Tarifs et remboursement, , Conventionné secte...   
999  [Tarifs et remboursement, , , , Conventionné, ...   

                                                card_1  \
991  [Carte et infor

In [178]:
#Pour rendre la localisation plus claire
def extract_lat_lng(map_location):
    try:
        location_dict = ast.literal_eval(map_location)
        return (location_dict['lat'], location_dict['lng'])
    except:
        return (None, None)

data_combined['Latitude'], data_combined['Longitude'] = zip(*data_combined['Map Location'].apply(extract_lat_lng))

data_combined.drop(columns = 'Map Location', inplace=True)


In [179]:
# Fonction pour traiter la colonne 'Skills'
def process_skills(skills):
    if skills == 'Skills missing':
        return np.nan
    else:
        return skills.split('\n')

data_combined['Skills Processed'] = data_combined['Skills'].apply(process_skills)

data_combined.drop(columns = 'Skills', inplace=True)

In [180]:
data_combined['Specialty'].value_counts()

Specialty
Chirurgien-dentiste                                                 152
Médecin généraliste                                                 104
Ophtalmologue                                                        75
Cabinet médical                                                      56
Spécialiste en chirurgie plastique reconstructrice et esthétique     40
                                                                   ... 
Chirurgien oral                                                       1
Acupuncteur                                                           1
Addictologue                                                          1
Chirurgien cancérologue                                               1
Diététicien                                                           1
Name: count, Length: 78, dtype: int64

In [181]:
data_combined['RPPS'] = data_combined['card_5'].apply(lambda x: x[2] if isinstance(x, list) and len(x) > 2 else np.nan)
data_combined.drop(columns = 'card_5', inplace = True)

In [182]:
data_combined['Nb Skills'] = data_combined['Skills Processed'].apply(lambda x: len(x)-1 if isinstance(x, list) else np.nan)

In [183]:
def paiement(card0):
    if isinstance(card0, list):
        if 'Moyens de paiement' in card0:
            return 1
        else:
            return 0
    else:
        return 0
data_combined['moyens_paiement'] = data_combined['card_0'].apply(paiement)


def vitale(card0):
    if isinstance(card0, list):
        if 'Carte Vitale acceptée' in card0:
            return 1
        elif 'Carte Vitale non acceptée' in card0:
            return 0
        else:
            return np.nan
    else:
        return np.nan
data_combined['carte_vitale'] = data_combined['card_0'].apply(paiement)


def secteur(card0):
    if isinstance(card0, list):
        a=np.nan
        for x in card0:
            if 'Conventionné secteur 2' in x:
                a=2
            elif 'Conventionné secteur 1' in x:
                a=1
            elif 'Conventionné secteur 3' in x:
                a=3
            elif 'Conventionné' in x:
                a=0
            else:
                a=a
        return a
    else:
        return np.nan
data_combined['secteur'] = data_combined['card_0'].apply(secteur)

In [184]:
data_combined['Présentation']=''
data_combined['Langues parlées']=''
data_combined['Diplômes nationaux et universitaires']=''
data_combined['Autres formations']=''
data_combined['Expériences']=''
data_combined['Travaux et publications']=''
data_combined['Prix et distinctions']=''
data_combined['site']=0

liste_mots_clefs = ['Présentation','Langues parlées','Diplômes nationaux et universitaires','Formations','Autres formations','Expériences', 'Site web', 'Travaux et publications', 'Prix et distinctions']

for i in range(data_combined.shape[0]):
    
    if isinstance(data_combined['card_2'].iloc[i],list) : 
        liste_card2 = data_combined['card_2'].iloc[i][1:]
        sous_liste = ''
        label = liste_mots_clefs[0]
        for x in liste_card2 :
            if x in liste_mots_clefs :
                data_combined.loc[i, label] = sous_liste
                label = x
                sous_liste = ''
            elif x=='Voir le site':
                data_combined.loc[i, 'site'] = 1
            elif x=='▾ Voir plus':
                None
            else :
                sous_liste = sous_liste+(x)

In [185]:
data_combined['Diplômes nationaux et universitaires b']=data_combined['Diplômes nationaux et universitaires'].apply(lambda x: 1 if len(x)>0 else 0)

In [186]:
data_combined['Nb caractères présentation'] = data_combined['Présentation'].apply(lambda x: len(x))

In [187]:
data_combined['Autres formations b']=data_combined['Autres formations'].apply(lambda x: 1 if len(x)>0 else 0)

In [188]:
data_combined['Nb langues']=0

for i in range(data_combined.shape[0]):
    if isinstance(data_combined['Langues parlées'].iloc[i],str) : 
        liste_langue = (data_combined['Langues parlées'].iloc[i]).split()
        count = 0
        for x in liste_langue :
            
            if  x != 'et'  :
                count +=1
        data_combined.loc[i,'Nb langues']=count

In [189]:
data_combined['Expériences b']=data_combined['Expériences'].apply(lambda x: 1 if len(x)>0 else 0)

In [190]:
liste1 = ["Carte et informations d'accès",'Moyens de transport','Parking public','Informations pratiques',"J'autorise le traitement d'informations (dont mon adresse IP) et leur transfert hors UE par Google Maps (USA) afin d’afficher la carte.En savoir plus", 'sur la collecte et le traitement des données par Google','AFFICHER LA CARTE']
def carac1(card1):
    nb=0
    for phrase in card1:
        if  phrase not in liste1:
            nb += len(phrase)
    return nb
data_combined['Carac_card_1'] = data_combined['card_1'].apply(lambda x: carac1(x)if isinstance(x, list) else np.nan)

In [191]:
def transport(card1):
    if isinstance(card1, list):
        if 'Moyens de transport' in card1:
            return 1
        else:
            return 0
    else:
        return 0
data_combined['Transport'] = data_combined['card_1'].apply(transport)

In [192]:
def parking(card1):
    if isinstance(card1, list):
        if 'Parking public' in card1:
            return 1
        else:
            return 0
    else:
        return 0
data_combined['Parking'] = data_combined['card_1'].apply(parking)

In [193]:
def infos(card1):
    if isinstance(card1, list):
        if 'Informations pratiques' in card1:
            return 1
        else:
            return 0
    else:
        return 0
data_combined['Infos'] = data_combined['card_1'].apply(infos)

In [194]:
def coordonnées(card3):
    if isinstance(card3, list):
        if 'Coordonnées' in card3:
            return 1
        else:
            return 0
    else:
        return 0
data_combined['Coordonnées'] = data_combined['card_3'].apply(coordonnées) 

In [195]:
def sansRDV(card3):
    if isinstance(card3, list):
        if 'Consultations sans rendez-vous' in card3:
            return 1
        else:
            return 0
    else:
        return 0
data_combined['Sans RDV'] = data_combined['card_3'].apply(sansRDV)

In [196]:
data_combined.drop(columns=['URL','card_1','card_2'],inplace=True)
data_combined.head()

,Doctor Name,Specialty,card_0,card_3,card_4,Latitude,Longitude,Skills Processed,RPPS,Nb Skills,...,Nb caractères présentation,Autres formations b,Nb langues,Expériences b,Carac_card_1,Transport,Parking,Infos,Coordonnées,Sans RDV
0,Dr Marian AGACHI,Neurochirurgien,NaN,"[Horaires et coordonnées, Horaires d'ouverture...",NaN,48.878328,2.431034,"[Expertises et actes, Neurochirurgie du rachis]",10001649291,1.0,...,748,0,3,0,154.0,1,0,1,1,0
1,Dr Anne Vaillant Moga,Médecin généraliste,"[Lieu 1, Lieu 2, Dr Anne Moga (Paris), 76 aven...","[Horaires et coordonnées, Horaires d'ouverture...","[Tarifs, Consultation préalable de médecine es...",48.869218,2.285468,"[Expertises et actes, Acné, Allergie, Apnée du...",10000378991,10.0,...,2538,1,2,1,180.0,1,0,1,0,0
2,Dr Gabriel Ohana,Chirurgien-dentiste,"[Tarifs et remboursement, , Carte Vitale accep...","[Horaires et coordonnées, Horaires d'ouverture...","[Tarifs, Consultation dentaire, 23 €, Traiteme...",48.894148,2.250698,"[Expertises et actes, Chirurgie buccale, Proth...",10003639373,3.0,...,481,1,1,0,240.0,1,0,1,1,0
3,Dr Thomas BAMBERGER,Dermatologue et vénérologue,"[Tarifs et remboursement, , , Conventionné sec...","[Horaires et coordonnées, Horaires d'ouverture...","[Tarifs, Consultation, 31,50 € à 56,50 €, Cont...",48.815433,2.343017,NaN,10003719571,NaN,...,407,0,1,1,155.0,1,0,1,1,0
4,Dr Layla Chatila,Chirurgien-dentiste,"[Tarifs et remboursement, , , , Conventionné, ...","[Horaires et coordonnées, Coordonnées, 01 45 2...","[Tarifs, Consultation dentaire, 23 €, Ces hono...",48.858804,2.274392,NaN,10005133003,NaN,...,246,0,1,0,202.0,1,1,1,1,0


In [199]:
data_combined[:,'nb carac card_0']=data_combined['card_0'].apply(lambda x: len(x))

TypeError: object of type 'float' has no len()

# Problème des nan

In [155]:
indices_nan = data_combined[data_combined['RPPS'].isna()].index.tolist()
print(indices_nan, len(indices_nan))

[8, 23, 27, 73, 74, 94, 112, 116, 117, 118, 121, 122, 123, 124, 127, 130, 131, 132, 135, 136, 138, 139, 140, 141, 142, 143, 145, 148, 150, 151, 152, 153, 156, 157, 158, 160, 161, 163, 166, 167, 169, 170, 172, 173, 175, 176, 177, 178, 179, 180, 182, 183, 184, 186, 187, 188, 190, 191, 192, 194, 196, 197, 198, 201, 202, 204, 209, 213, 214, 215, 216, 219, 220, 221, 222, 223, 225, 226, 227, 229, 230, 231, 233, 234, 235, 236, 238, 239, 240, 241, 245, 246, 247, 249, 250, 252, 254, 256, 259, 260, 262, 263, 264, 265, 267, 268, 270, 271, 272, 276, 277, 278, 279, 280, 281, 282, 284, 285, 286, 287, 288, 289, 290, 291, 292, 293, 295, 296, 297, 298, 299, 300, 301, 302, 303, 304, 305, 306, 307, 308, 309, 310, 312, 315, 316, 317, 318, 319, 320, 321, 323, 324, 325, 326, 327, 328, 329, 331, 335, 337, 339, 344, 346, 347, 348, 349, 350, 351, 352, 353, 354, 356, 358, 359, 360, 365, 367, 368, 371, 372, 373, 374, 376, 378, 379, 380, 381, 382, 383, 384, 387, 388, 390, 392, 393, 395, 396, 397, 398, 399, 400, 4

In [156]:
data['Profile Cards'].iloc[999]

'{\'card_0\': \'Tarifs et remboursement\\n\\n\\n\\nConventionné\\nCarte Vitale acceptée\\nTiers payant : Sécurité sociale\\nVoir les tarifs\\nMoyens de paiement\\nChèques, espèces et carte bancaire\\nExpertises et actes\\nBilan podologique\\nIonophorèse\\nK-Tape\\nKinesio Taping\\nOrthonyxie\\nOrthoplastie\\nOrthèse plantaire\\nPodologie\\nPédicurie\\nSemelles orthopédiques\\nVerrue\', \'card_1\': "Carte et informations d\'accès\\n9 Avenue Ampère, 77420 Champs-sur-Marne\\nMoyens de transport\\nBus - Ampere (lignes 213 et 312)\\nBus - Noisy-Champs Rer - Descartes (ligne 212)\\nInformations pratiques\\n1er étage avec ascenseur\\nEntrée accessible\\nJ\'autorise le traitement d\'informations (dont mon adresse IP) et leur transfert hors UE par Google Maps (USA) afin d’afficher la carte.En savoir plus\\nsur la collecte et le traitement des données par Google\\nAFFICHER LA CARTE", \'card_2\': "Présentation\\nDepuis janvier 2010, je vous reçois dans mon cabinet à Champs-sur-Marne à deux pas de

In [157]:
data_combined_sans_nan = data_combined[data_combined['RPPS'].notna()]
data_combined_sans_nan

,Doctor Name,Specialty,card_0,card_3,card_4,Latitude,Longitude,Skills Processed,RPPS,Nb Skills,...,Nb caractères présentation,Autres formations b,Nb langues,Expériences b,Carac_card_1,Transport,Parking,Infos,Coordonnées,Sans RDV
0,Dr Marian AGACHI,Neurochirurgien,NaN,"[Horaires et coordonnées, Horaires d'ouverture...",NaN,48.878328,2.431034,"[Expertises et actes, Neurochirurgie du rachis]",10001649291,1.0,...,748,0,3,0,154.0,1,0,1,1,0
1,Dr Anne Vaillant Moga,Médecin généraliste,"[Lieu 1, Lieu 2, Dr Anne Moga (Paris), 76 aven...","[Horaires et coordonnées, Horaires d'ouverture...","[Tarifs, Consultation préalable de médecine es...",48.869218,2.285468,"[Expertises et actes, Acné, Allergie, Apnée du...",10000378991,10.0,...,2538,1,2,1,180.0,1,0,1,0,0
2,Dr Gabriel Ohana,Chirurgien-dentiste,"[Tarifs et remboursement, , Carte Vitale accep...","[Horaires et coordonnées, Horaires d'ouverture...","[Tarifs, Consultation dentaire, 23 €, Traiteme...",48.894148,2.250698,"[Expertises et actes, Chirurgie buccale, Proth...",10003639373,3.0,...,481,1,1,0,240.0,1,0,1,1,0
3,Dr Thomas BAMBERGER,Dermatologue et vénérologue,"[Tarifs et remboursement, , , Conventionné sec...","[Horaires et coordonnées, Horaires d'ouverture...","[Tarifs, Consultation, 31,50 € à 56,50 €, Cont...",48.815433,2.343017,NaN,10003719571,NaN,...,407,0,1,1,155.0,1,0,1,1,0
4,Dr Layla Chatila,Chirurgien-dentiste,"[Tarifs et remboursement, , , , Conventionné, ...","[Horaires et coordonnées, Coordonnées, 01 45 2...","[Tarifs, Consultation dentaire, 23 €, Ces hono...",48.858804,2.274392,NaN,10005133003,NaN,...,246,0,1,0,202.0,1,1,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
990,Dr Danielle Taconet,Ophtalmologue,"[Tarifs et remboursement, , , Conventionné sec...","[Horaires et coordonnées, Contact d'urgence, E...",NaN,49.060704,2.184674,"[Expertises et actes, Champ visuel, Laser opht...",10003749230,4.0,...,7800,0,2,0,231.0,1,0,1,1,0
994,Dr Stella Danan-Hadida,Gynécologue médicale et obstétrique,"[Tarifs et remboursement, , , , Conventionné s...","[Horaires et coordonnées, Horaires d'ouverture...","[Tarifs, Consultation vidéo, 58 €, Consultatio...",48.871909,2.346805,"[Expertises et actes, Gynécologie obstétrique,...",10000292291,7.0,...,626,0,1,0,249.0,1,1,1,1,0
995,Dr Philippe Lazare SELLAM,Spécialiste en chirurgie plastique reconstruct...,"[Lieu 1, Lieu 2, Cabinet du Docteur Philippe L...","[Horaires et coordonnées, Contact d'urgence, E...","[Tarifs, Consultation, 23 € à 120 €, Exérèse d...",48.868676,2.325944,NaN,10000558964,NaN,...,535,1,1,0,271.0,1,1,1,1,0
996,Dr Michèle Maizil,Chirurgien-dentiste,NaN,"[Horaires et coordonnées, Horaires d'ouverture...",NaN,48.834970,2.246637,NaN,10000202985,NaN,...,371,0,0,0,376.0,1,1,1,1,0


# Ouverture base de données activités

In [158]:
annuaire = pd.read_csv('data/PS_LibreAcces_Personne_activite_small.csv')
annuaire.shape

/var/folders/rf/dqdwsk6s06ngm9b4j17fkb2r0000gn/T/ipykernel_85055/2401431438.py:1: DtypeWarning: Columns (10,15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  annuaire = pd.read_csv('data/PS_LibreAcces_Personne_activite_small.csv')


(83743, 27)

In [159]:
annuaire.isnull().sum()

identifiantpp                           0
identificationnationalepp               0
codecivilité                            0
nomdexercice                            1
prénomdexercice                         1
codecatégorieprofessionnelle            0
codesavoirfaire                         0
libellésavoirfaire                      0
codemodeexercice                        0
numérosiretsite                     58015
numérofinesssite                    57014
identifianttechniquedelastructur        0
raisonsocialesite                       0
numérovoiecoordstructure            10187
indicerépétitionvoiecoordstructu    78376
codetypedevoiecoordstructure        57627
libellétypedevoiecoordstructure     57627
libellévoiecoordstructure            1195
bureaucedexcoordstructure               0
codepostalcoordstructure                0
codecommunecoordstructure               1
libellécommunecoordstructure            1
libellépayscoordstructure           26729
codesecteurdactivité              

In [160]:
annuaire['identifiantpp']=annuaire['identifiantpp'].astype(str)

# Merge doctolib généraliste et base de données

In [161]:
medecin_gen = data_combined_sans_nan[data_combined_sans_nan['Specialty']=='Médecin généraliste']

In [162]:
liste_med = annuaire.merge(medecin_gen,how='outer',left_on='identifiantpp',right_on='RPPS')

In [163]:
print(liste_med.shape[0]-annuaire.shape[0], medecin_gen.shape[0],annuaire.shape[0])

0 39 83743


In [164]:
liste_med['codepostalcoordstructure']

0        97139
1        97110
2        97133
3        97110
4        97136
         ...  
83738    94456
83739    93370
83740    94275
83741    40024
83742    94195
Name: codepostalcoordstructure, Length: 83743, dtype: int64

In [165]:
def departement(codepostal):
    code = int(codepostal)/1000
    if int(code) == 97 :
        return(str(int(codepostal/100)))
    if int(code) < 10 :
        return('0'+str(int(code)))
    return (str(int(code)))

In [166]:
liste_med.loc[:, 'code dépar'] = liste_med['codepostalcoordstructure'].apply(departement)
liste_med.shape

(83743, 62)

Ouverture concurrence d'après INSEE

In [167]:
concu = pd.read_excel('data/TCRD_068.xlsx')
concu.rename(columns={'Professionnels de santé au 1ᵉʳ janvier 2023 : comparaisons départementales' : 'Code Département','Unnamed: 1' : 'Nom Département','Unnamed: 2':'Nombre de médecins','Unnamed: 3':'Nombre de médecins pour 100 000 habitants','Unnamed: 4':'généralistes','Unnamed: 5':'spécialistes','Unnamed: 6':'dentistes','Unnamed: 7':'Pharm.'}, inplace=True)
concu.drop(labels = [0,1,2,3,108],axis=0, inplace=True)
concu

,Code Département,Nom Département,Nombre de médecins,Nombre de médecins pour 100 000 habitants,généralistes,spécialistes,dentistes,Pharm.
4,01,Ain,1162,174,99,75,53,78
5,02,Aisne,1107,211,99,111,45,91
6,03,Allier,917,275,134,141,51,111
7,04,Alpes-de-Haute-Provence,483,291,165,125,54,103
8,05,Hautes-Alpes,705,503,291,213,72,135
...,...,...,...,...,...,...,...,...
103,972,Martinique,1171,332,152,180,68,99
104,973,Guyane,717,242,123,120,30,46
105,974,La Réunion,3170,364,176,189,67,94
106,976,Mayotte,265,89,49,39,9,31


In [168]:
liste_med = liste_med.merge(concu[['Code Département','généralistes']], left_on='code dépar', right_on='Code Département',how='inner')

In [169]:
liste_med

,identifiantpp,identificationnationalepp,codecivilité,nomdexercice,prénomdexercice,codecatégorieprofessionnelle,codesavoirfaire,libellésavoirfaire,codemodeexercice,numérosiretsite,...,Expériences b,Carac_card_1,Transport,Parking,Infos,Coordonnées,Sans RDV,code dépar,Code Département,généralistes
0,10000003870,810000003870,M,VUILLET,MICHEL,C,SM26,Qualifié en Médecine Générale,S,2.697104e+13,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,971,971,150
1,10000003870,810000003870,M,VUILLET,MICHEL,C,SM26,Qualifié en Médecine Générale,L,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,971,971,150
2,10000004688,810000004688,M,WEILL,EDGARD,C,SM26,Qualifié en Médecine Générale,L,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,971,971,150
3,10000005180,810000005180,M,BORDESSOULES,PIERRE,C,SM54,Médecine Générale,L,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,971,971,150
4,10000005537,810000005537,M,DORVILLE,ALBERT,C,SM53,Spécialiste en Médecine Générale,L,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,971,971,150
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83052,10110318689,810110318689,MME,NABTI,Dahlia,C,SM53,Spécialiste en Médecine Générale,S,2.675005e+13,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,94,94,122
83053,10110318705,810110318705,MME,LAOUISSETTE,Meriem,C,SM53,Spécialiste en Médecine Générale,S,2.000180e+13,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,93,93,99
83054,10110318713,810110318713,M,RAZAFIMANDIMBY,Jean-Marc,C,SM53,Spécialiste en Médecine Générale,S,2.675005e+13,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,94,94,122
83055,10110318739,810110318739,MME,DAHMANI,Kamélia,C,SM53,Spécialiste en Médecine Générale,S,2.640043e+13,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40,40,173


In [170]:
liste_med.columns

Index(['identifiantpp', 'identificationnationalepp', 'codecivilité',
       'nomdexercice', 'prénomdexercice', 'codecatégorieprofessionnelle',
       'codesavoirfaire', 'libellésavoirfaire', 'codemodeexercice',
       'numérosiretsite', 'numérofinesssite',
       'identifianttechniquedelastructur', 'raisonsocialesite',
       'numérovoiecoordstructure', 'indicerépétitionvoiecoordstructu',
       'codetypedevoiecoordstructure', 'libellétypedevoiecoordstructure',
       'libellévoiecoordstructure', 'bureaucedexcoordstructure',
       'codepostalcoordstructure', 'codecommunecoordstructure',
       'libellécommunecoordstructure', 'libellépayscoordstructure',
       'codesecteurdactivité', 'libellésecteurdactivité', 'coderôle',
       'libellérôle', 'Doctor Name', 'Specialty', 'card_0', 'card_3', 'card_4',
       'Latitude', 'Longitude', 'Skills Processed', 'RPPS', 'Nb Skills',
       'moyens_paiement', 'carte_vitale', 'secteur', 'Présentation',
       'Langues parlées', 'Diplômes nationaux

In [171]:
data_final = data_avec_concu[['nomdexercice', 'prénomdexercice','RPPS','Specialty','card_0', 'card_3', 'card_4','Skills Processed','Nb Skills', 'moyens_paiement',
       'carte_vitale', 'secteur', 'Présentation', 'Langues parlées',
       'Diplômes nationaux et universitaires', 'Autres formations',
       'Expériences', 'Travaux et publications',
       'Prix et distinctions', 'site', 'Formations',
       'Diplômes nationaux et universitaires b', 'Nb caractères présentation',
       'Autres formations b', 'Nb langues', 'Expériences b', 'Carac_card_1',
       'Transport', 'Parking', 'Infos', 'Coordonnées', 'Sans RDV','Code Département', 'Nom Département',
       'Nombre de médecins', 'Nombre de médecins pour 100 000 habitants',
       'généralistes', 'spécialistes','dentistes']]
data_final.head()

,nomdexercice,prénomdexercice,RPPS,Specialty,card_0,card_3,card_4,Skills Processed,Nb Skills,moyens_paiement,...,Infos,Coordonnées,Sans RDV,Code Département,Nom Département,Nombre de médecins,Nombre de médecins pour 100 000 habitants,généralistes,spécialistes,dentistes
0,LAZZARESCHI,JULIE,069204550,Orthoptiste,"[Tarifs et remboursement, , , , Conventionné, ...","[Horaires et coordonnées, Horaires d'ouverture...","[Tarifs, Bilan orthoptique, 39 €, Bilan orthop...","[Expertises et actes, Bilan neurovisuel, Bilan...",5.0,1.0,...,1.0,1.0,0.0,06,Alpes-Maritimes,5095,461,173,288,123
1,NALLET,EMMANUEL,10000030766,ORL - Chirurgien de la face et du cou,NaN,"[Horaires et coordonnées, Horaires d'ouverture...",NaN,"[Expertises et actes, ORL et otologie, ORL péd...",4.0,0.0,...,0.0,1.0,0.0,971,Guadeloupe,1209,319,150,169,62
2,ARNAUD,LOUIS,10000058080,Chirurgien-dentiste,NaN,"[Horaires et coordonnées, Horaires d'ouverture...",NaN,"[Expertises et actes, Apnée du sommeil, Chirur...",8.0,0.0,...,1.0,1.0,0.0,92,Hauts-de-Seine,6238,381,135,246,98
3,SALA,BERNARD,10000069749,Chirurgien-dentiste,"[Tarifs et remboursement, , Carte Vitale accep...",NaN,"[Tarifs, Consultation dentaire, 23 €, Detartra...","[Expertises et actes, Chirurgie buccale, Proth...",2.0,1.0,...,1.0,0.0,0.0,75,Paris,18805,888,234,654,152
4,COHEN,THIERRY,10000076546,Chirurgien-dentiste,"[Tarifs et remboursement, , , , Conventionné, ...","[Horaires et coordonnées, Horaires d'ouverture...",NaN,"[Expertises et actes, Blanchiment des dents, C...",11.0,1.0,...,1.0,1.0,0.0,75,Paris,18805,888,234,654,152


Ouverture BAN

In [ ]:
ban = pd.read_csv('data/adresses-france.csv')
ban.head()